In [ ]:
import os
import sys
import torch

project_dir = os.path.join(os.getcwd(),'..')
if project_dir not in sys.path:
    sys.path.append(project_dir)
from experiments.MNIST import ExperimentMNIST



from experiments.MNIST import ExperimentMNIST
from matplotlib import pyplot as plt
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler




exp = ExperimentMNIST(0.01, 0, seed=None)

# Create a model
model = OneClassSVM(kernel='rbf', nu=0.05, gamma='auto')
x_train, y_train = zip(*[(_x, _y) for _x, _y in exp.train_dataset])
x_train, y_train = torch.stack(x_train), torch.tensor(y_train)

x_train = x_train.reshape(-1, 28*28)
_x_train = x_train.numpy()

scaler = StandardScaler()
_x_train = scaler.fit_transform(_x_train)


# Fit the model
model.fit(_x_train)

# Predict
x_test, y_test = zip(*[(_x, _y) for _x, _y in exp.test_dataset])
x_test, y_test = torch.stack(x_test), torch.tensor(y_test)
x_test = x_test.reshape(-1, 28*28)
_x_test = scaler.transform(x_test)
y_pred = model.predict(_x_test)

# Evaluate
from sklearn.metrics import roc_curve, auc
fpr, tpr, _ = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)

# Plot
with plt.style.context(("seaborn-colorblind")):
    fig = plt.figure(figsize=(6, 4))
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False Positive Rate', fontsize='x-large')
    plt.ylabel('True Positive Rate', fontsize='x-large')
    plt.legend(loc="lower right")

    plt.tick_params(axis='both', which='major', labelsize='large')

    plt.show()

In [ ]:
# Cross validation using training set for obtaining the best hyperparameters
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': np.logspace(-3, 1, 10),
                     'nu': [0.1, 0.2, 0.3, 0.4, 0.5]}]

scores = ['roc_auc_ovr']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(
        OneClassSVM(), tuned_parameters, scoring=score
    )
    clf.fit(x_train.numpy())

    # print("Best parameters set found on development set:")
    # print()
    # print(clf.best_params_)
    # print()
    # print("Grid scores on development set:")
    # print()
    # means = clf.cv_results_['mean_test_score']
    # stds = clf.cv_results_['std_test_score']
    # for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    #     print("%0.3f (+/-%0.03f) for %r"
    #           % (mean, std * 2, params))
    # print()

    # print("Detailed classification report:")
    # print()
    # print("The model is trained on the full development set.")
    # print("The scores are computed on the full evaluation set.")
    # print()
    # y_true, y_pred = y_test, clf.predict(x_test.numpy())
    # # print(classification_report(y_true, y_pred))
    # # print()